In [2]:
import sys
sys.path.append('../../lib')

from readmdict import MDX
from bs4 import BeautifulSoup
import re
import json
from jinja2 import Environment, FileSystemLoader

mdx = MDX('./韦氏高阶英汉双解词典.mdx') # change to your own path

In [8]:
dict = {}
r = re.compile(r'@@@LINK=(.*)')
for i in mdx.items():
    entry = i[0].decode("utf-8")
    content = i[1].decode("utf-8")
    if "@@@LINK" in content:
        matches = r.search(content)
        e = matches.group(1).strip()
        if e in dict:
            dict[e]["index"].append(entry)
        else:
            dict[e] = {
                "index": [entry],
                "html": ""
            }
    else:
        dom = BeautifulSoup(content, "html.parser")
        
        for tag in dom.find_all(True):
            # Get the class attribute, if it exists
            class_attr = tag.get('class')

            # Clear all attributes
            tag.attrs = {}

            # Re-add the class attribute if it existed
            if class_attr:
                tag['class'] = class_attr


        if entry in dict:
            dict[entry]['html'] = str(dom)
            dict[entry]['index'].append(entry)
        else:
            dict[entry] = {
                "index": [entry],
                "html": str(dom)
            }

with open("./dict.json", "w", encoding="utf-8") as f:
    json.dump(dict, f, ensure_ascii=False)

In [ ]:
data = {}
with open('./dict.json', 'r') as file:
    data = json.load(file)

print(data["don't"]['html'])

template_env = Environment(loader=FileSystemLoader('.'))
template = template_env.get_template('template.html.jinja')
rendered = template.render({"entries":data})
with open("./mwalecd.xml", "w") as f:
    f.write(rendered)